# Load Horizons Data into KeplerDB

In [1]:
# Core
import numpy as np
import pandas as pd

# Utility
import re
import glob
import time

# Database
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
# MSE imports
import kepler_sieve
from horizons_files import hrzn_txt2csv, hrzn_csv2df, hrzn_df2db, hrzn_txt2db, hrzn_csv2db, hrzn_load
from asteroid_element import load_data_numbered, load_data_unnumbered, load_data_impl
import db_config

## Load all the Horizons CSV Files into JPL.HorizonsImport

In [ ]:
# hrzn_load()

## Testing - Load CSV, Display as DataFrame

In [ ]:
fname_txt = '../data/jpl/horizons/planets/010_sun.txt'
# fname_txt = '../data/jpl/horizons/planets/001_mercury_barycenter.txt'
# fname_txt = '../data/jpl/horizons/moons/301_moon.txt'
# fname_txt = '../data/jpl/horizons/asteroids/ast_0001.txt'

In [ ]:
fname_csv = hrzn_txt2csv(fname_txt)

In [ ]:
fname_csv

In [ ]:
df = hrzn_csv2df(fname_csv)

In [ ]:
pd.set_option('display.max_rows', 20)

In [ ]:
df

## Testing - Populate Database from CSV

In [ ]:
connection_str = f'mysql+pymysql://{db_config.username}:{db_config.password}@{db_config.hostname}/JPL'

In [ ]:
engine = create_engine(connection_str)

engine.table_names()

In [ ]:
type(engine.connect())

In [ ]:
sqlalchemy.engine.Connection

In [ ]:
sql = \
f"""
load data infile '{fname_csv}'
into table JPL.HorizonsImport 
fields terminated by ','
lines terminated by '\n'
ignore 1 lines
(BodyTypeCD, BodyNumber, BodyName, IntegrationSource, JD, CalendarDateTime, delta_T, qx, qy, qz, vx, vy, vz)
set HorizonsImportID = NULL;
"""

In [ ]:
sql

In [ ]:
print(sql)

In [ ]:
sql = 'select * from JPL.HorizonsImport limit 10;'

In [ ]:
with engine.connect() as conn:
    conn.execute(sql)

In [ ]:
# hrzn_txt2db(fname_txt)

## Load the Asteroid Orbital Elements from JPL

In [4]:
def load_data_numbered() -> pd.DataFrame:
    """Load the asteroid data for numbered asteroids into a Pandas DataFrame"""
    # The source for this file is at https://ssd.jpl.nasa.gov/?sb_elem
    fname: str = '../data/jpl/orbital_elements/asteroid_numbered.txt'

    # The field names in the JPL file and their column positions
    names: List[str] = ['Num', 'Name', 'Epoch', 'a', 'e', 'i', 'w', 'Node', 'M', 'H', 'G', 'Ref']
    colspec_tbl: Dict[str, Tuple[int, int]] = {
        'Num': (0,6), 
        'Name': (7, 25), 
        'Epoch': (25, 30), 
        'a': (31, 41), 
        'e': (42, 52), 
        'i': (54, 62), 
        'w': (63, 72),
        'Node': (73, 82),
        'M': (83, 94),
        'H': (95, 100),
        'G': (101, 105),
        'Ref': (106, 113),
    }
    
    # Other arguments for Pandas file import
    colspecs: List[Tuple[int, int]] = [colspec_tbl[nm] for nm in names]
    header: int = 0
    skiprows: List[int] = [1]
    dtype: Dict[str, int] = {
        'Num': int,
        'Name': str,
        'Epoch': float,
        'a': float,
        'e': float,
        'i': float,
        'w': float,
        'Node': float,
        'M': float,
        'H': float,
        'G': float,
        'Ref': str,
    }

    # Read the DataFrame
    df: pd.DataFrame = pd.read_fwf(fname, colspecs=colspecs, header=header, names=names, skiprows=skiprows, dtype=dtype)
    # Set the asteroid number field to be the index
    df.set_index(keys=['Num'], drop=False, inplace=True)
    return df

In [8]:
load_data_numbered()

,Num,Name,Epoch,a,e,i,w,Node,M,H,G,Ref
Num,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,10.59407,73.59769,80.30553,77.372097,3.40,0.12,JPL 47
2,2,Pallas,59000.0,2.773841,0.229972,34.83293,310.20239,173.02474,144.975675,4.20,0.11,JPL 37
3,3,Juno,59000.0,2.668285,0.256936,12.99104,248.06619,169.85148,125.435355,5.33,0.32,JPL 112
4,4,Vesta,58600.0,2.361418,0.088721,7.14177,150.72854,103.81080,95.861938,3.00,0.32,JPL 35
5,5,Astraea,59000.0,2.574037,0.190913,5.36743,358.64842,141.57103,17.846343,6.90,0.15,JPL 114
6,6,Hebe,59000.0,2.424533,0.203219,14.73965,239.73627,138.64343,190.686496,5.80,0.24,JPL 89
7,7,Iris,59000.0,2.387375,0.230145,5.52160,145.20154,259.56394,247.425812,5.60,0.15,JPL 110
8,8,Flora,59000.0,2.201415,0.155833,5.88908,285.45892,110.87652,315.318013,6.50,0.28,JPL 118
9,9,Metis,59000.0,2.386189,0.123300,5.57649,6.33732,68.90946,23.912204,6.30,0.17,JPL 116


In [12]:
load_data_unnumbered()

,Num,Name,Epoch,a,e,i,w,Node,M,H,G,Ref
Num,,,,,,,,,,,,
1000001,1000001,1927 LA,25051.0,3.344071,0.333618,17.63150,341.10951,191.71742,45.720173,11.00,0.15,JPL 10
1000002,1000002,1935 UZ,28097.0,2.157511,0.251290,4.75775,281.18859,134.51847,342.396110,99.00,0.00,JPL 5
1000003,1000003,1937 CK,59000.0,2.320184,0.137423,6.54805,130.26997,273.87195,300.574592,99.00,0.00,JPL 5
1000004,1000004,1939 RR,29540.0,2.904674,0.253684,13.13929,64.83753,322.99733,324.193410,12.00,0.15,MPC06878
1000005,1000005,1942 RH,30612.0,2.257215,0.145324,4.22398,134.53239,205.85775,3.933424,13.80,0.15,JPL 3
1000006,1000006,1960 SB1,37203.0,3.008002,0.530847,10.41063,218.63151,201.22482,345.317604,17.22,0.15,JPL 3
1000007,1000007,1979 MW5,44051.0,3.188524,0.297586,9.39365,161.05051,124.45700,3.480559,16.05,0.15,JPL 5
1000008,1000008,1979 XB,44221.0,2.228139,0.708457,24.73433,75.57980,86.05549,346.318937,18.60,0.15,JPL 12
1000009,1000009,1981 EG6,44666.0,2.906377,0.298918,6.54614,194.97729,295.94267,23.665573,17.27,0.15,JPL 4


In [6]:
# df = load_data_impl()